Presentado por:<br>
David Crespo <br>
Cristina Mejia <br>
Melissa Fuentes <br>

El objeto de este proyecto es alicar los conceptos de aprendizaje de máquina vistos en el curso. Se hará el entrenamiento de modelos de clasificación usando árboles de decisión, naive bayes y redes neuronales para resolver un problema de clasificación a partir de un conjunto de datos relacionados a enfermedades del corazón.


Primero se importan las librerías:

In [ ]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

Ahora cargamos los dos archivos de conjuntos de datos.

In [ ]:
heartDataCSV = pd.read_csv("../input/heart-disease-uci/heart.csv")
#heartDiseaseData = pd.read_csv("HeartDisease.csv")

print(heartDataCSV.columns)

Se convierten los datos en un dataFrame de Pandas

In [ ]:
heartData = pd.DataFrame(heartDataCSV)

Histogramas

In [ ]:


heartData["age"].hist(figsize=(6,3))
plt.title('Age')
plt.show()
heartData["trestbps"].hist(figsize=(6,3))
plt.title('Trestbps')
plt.show()
heartData["chol"].hist(figsize=(6,3))
plt.title('Chol')
plt.show()
heartData["thalach"].hist(figsize=(6,3))
plt.title('Thalach')
plt.show()
heartData["oldpeak"].hist(figsize=(6,3))
plt.title('Oldpeak')
plt.show()
heartData["ca"].hist(figsize=(6,3))
plt.title('Ca')
plt.show()



In [ ]:
sums = heartData.sex.sum()
print(sums)

Diagramas de pastel

In [ ]:
heartData.restecg.groupby(heartData.sex).sum().plot(kind = 'pie')
plt.axis('equal')
plt.ylabel('genre')
plt.show()
heartData.restecg.groupby(heartData.fbs).sum().plot(kind = 'pie')
plt.axis('equal')
plt.ylabel('fbs')
plt.show()
heartData.sex.groupby(heartData.restecg).sum().plot(kind = 'pie')
plt.axis('equal')
plt.ylabel('restecg')
plt.show()
heartData.restecg.groupby(heartData.exang).sum().plot(kind = 'pie')
plt.axis('equal')
plt.ylabel('exang')
plt.show()
heartData.restecg.groupby(heartData.thal).sum().plot(kind = 'pie')
plt.axis('equal')
plt.ylabel('thal')
plt.show()


**Organización de conjunto de datos**

A continuación se analaizará el conjunto de datos para considerar si hace falta completar o remover elementos faltantes o nulos

In [ ]:
#Punto 5 en el que miramos si hay valores nulos o faltantes en nuestro conjunto de datos
heartDataCSV = pd.read_csv("../input/heart-disease-uci/heart.csv")
x = pd.notnull(heartDataCSV)  
heartDataCSV[x]

#Gracias a la función notnull, podemos mostrar todo nuestro conjunto de datos cuya celda sea distinta de NaN, y como nos retorna las 303 filas en total con las 14 columnas, podemos decir que nuestro conjunto de datos no tenia regitros faltantes




Gracias a la función notnull, podemos mostrar todo nuestro conjunto de datos cuya celda sea distinta de NaN, y como nos retorna las 303 filas en total con las 14 columnas, podemos decir que nuestro conjunto de datos no tenia regitros faltantes, por lo que concluimos que no hubo necesidad de realizar ninguna modificación al conjunto de datos.

Ahora a continuación partiremos los datos en dos grupos. El 80% de ellos para entrenamiento y el 20% para pruebas.


In [ ]:
from sklearn.model_selection import train_test_split


# Create a new array with the added features: features_two
variables = heartData[['age','sex','cp','trestbps', 'chol', 'fbs', 'restecg',
                      'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']].values
target = heartData['target'].values


# Split the data into train and test
trainX, testX, trainY, testY = train_test_split(variables, target, test_size=0.2)
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

**Naive Bayes**

In [ ]:
#Entrenamos un modelo de Naive Bayes

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix


gnb = GaussianNB()

modeloBayes = gnb.fit(trainX, trainY)

#Mostramos el score del nuevo arbol de desición 
print("Exactitud en el entrenamiento")
print(modeloBayes.score(trainX, trainY))

print("Exactitud en la prueba")
print(modeloBayes.score(testX, testY))

#Mostramos la matriz de confusión para el modelo 
confusion_matrix(modeloBayes.predict(testX), testY)



**Árboles de decisión**

In [ ]:
#Arbolito bonito

arbol = DecisionTreeClassifier(max_depth = 4, min_samples_split =4)
arbol.fit(trainX, trainY)

#Print the score on the train data
print(arbol.score(trainX, trainY))
#Print the score on the test data
print(arbol.score(testX, testY))

In [ ]:
export_graphviz(arbol, out_file='arbol.dot',impurity=False, filled=True)

In [ ]:
import pydotplus
from IPython.display import Image

pydot_graph = pydotplus.graph_from_dot_file("arbol.dot")
Image(pydot_graph.create_png())


In [ ]:
#Matriz de confusión
from sklearn.metrics import confusion_matrix

confusion_matrix(arbol.predict(testX), testY)

**Red neuronal**

Utilizaremos numpy para el manejo de arrays de Keras, se importa el  modelo Sequential y la capa Dense

Creamos los arrays de entrada seran trainX y trainY

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=13, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

se crea un modelo vació de tipo Sequential, esto quiere decir que crearemos una serie de capas

Con esto indicamos el tipo de pérdida (loss) que utilizaremos, el «optimizador» de los pesos de las conexiones de las neuronas y las métricas que queremos obtener.

Indicamos con model.fit() las entradas y sus salidas y la cantidad de iteraciones de 

In [ ]:

 
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])
 
model.fit(trainX, trainY, epochs=1000)
 
# evaluamos el modelo
scores = model.evaluate(testX, testY)
 
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print (model.evaluate(trainX,trainY))

#Mostramos la matriz de confusión para el modelo 
confusion_matrix(model.predict_classes(testX), testY)

al finalizar, los resultados permite ver el nivel de acierto de la primera y segunda iteración 

Luego en la «epoch» se hacen los ajustes correspondientes a la red

